### Input
Set either INPUT_FILE or INPUT_TEXT variable.

In [ ]:
#pip install -r requirements.txt

In [11]:
# Provide either INPUT_FILE path or INPUT_TEXT to summarize.
INPUT_FILE="" # Insert file path here
INPUT_TEXT="""Insert text to summarize here."""

# Style of summarization:

# Numbered List style
STYLE="Return your response as numbered list which covers the main points of the text."
PROMPT_TRIGGER="NUMBERED LIST SUMMARY"

# One sentence style
# STYLE="Return your response as one sentence which covers the main points of the text.",
# PROMPT_TRIGGER="ONE SENTENCE SUMMARY",

# Concise style
# STYLE="Return your response as concise summary which covers the main points of the text.",
# PROMPT_TRIGGER="CONCISE SUMMARY",

# Detailed style
# STYLE="Return your response as detailed summary which covers the main points of the text and key facts and figures.",
# PROMPT_TRIGGER="DETAILED SUMMARY",

# Output language, try e.g. Polish, Spanish, etc 
OUTPUT_LANGUAGE = "English"

# Should output verbose info from underlying models, etc.
VERBOSE=True

### Model params & setup

In [6]:
# Model file
MODEL_FILE="./models/mistral-7b-openorca.Q5_K_M.gguf"

MODEL_CONTEXT_WINDOW=8192

# Maximal lenght of model's output, in tokens.
MAX_ANSWER_TOKENS = 2048

# Chunk params in characters (not tokens).
CHUNK_SIZE=10000
CHUNK_OVERLAP=500

In [7]:
from langchain.llms import LlamaCpp

llm = LlamaCpp(
    model_path=MODEL_FILE,
    n_ctx=MODEL_CONTEXT_WINDOW,
    # Maximal lenght of model's output, in tokens.
    max_tokens=MAX_ANSWER_TOKENS,
    # Don't be creative.
    temperature=0,
    verbose=VERBOSE,

    # Remove next two lines if NOT using macOS & M1 processor:
    n_batch=512,
    n_gpu_layers=1,
)

AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


### Implementation

In [8]:
from langchain.document_loaders import TextLoader

def load_content():
    """Loads INPUT_FILE if set, otherwise returns INPUT_TEXT"""

    if INPUT_FILE:
        if INPUT_FILE.endswith(".pdf"):
            loader = PyPDFLoader(INPUT_FILE)
            docs = loader.load()
            print(f"PDF: loaded {len(docs)} pages")
            return "\n".join([d.page_content for d in docs])
        
        docs =  TextLoader(INPUT_FILE).load()
        return docs[0].page_content

    return INPUT_TEXT


In [9]:
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

combine_prompt_template = """
Write a summary of the following text delimited by tripple backquotes.
{style}

```{content}```

{trigger} in {language}:
"""

map_prompt_template = """
Write a concise summary of the following:
{text}

CONCISE SUMMARY in {language}:
"""

def summarize_base(llm, content):
    """Summarize whole content at once. The content needs to fit into model's context window."""

    prompt = PromptTemplate.from_template(
        combine_prompt_template
    ).partial(
        style=STYLE,
        trigger=PROMPT_TRIGGER,
        language=OUTPUT_LANGUAGE,
    )

    chain = LLMChain(llm=llm, prompt=prompt, verbose=VERBOSE)
    output = chain.run(content)

    return output


def summarize_map_reduce(llm, content):
    """Summarize content potentially larger that model's context window using map-reduce approach."""

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
    )

    split_docs = text_splitter.create_documents([content])
    print(f"Map-Reduce content splits ({len(split_docs)} splits): {[len(sd.page_content) for sd in split_docs]}")

    map_prompt = PromptTemplate.from_template(
        map_prompt_template
    ).partial(
        language=OUTPUT_LANGUAGE,
    )
    
    combine_prompt = PromptTemplate.from_template(
        combine_prompt_template
    ).partial(
        style=STYLE,
        trigger=PROMPT_TRIGGER,
        language=OUTPUT_LANGUAGE,
    )

    chain = load_summarize_chain(
        llm=llm,
        chain_type="map_reduce",
        map_prompt=map_prompt,
        combine_prompt=combine_prompt,
        combine_document_variable_name="content",
        verbose=VERBOSE,
    )

    output = chain.run(split_docs)
    return output

### Main program

In [13]:
%%time 

INPUT_TEXT = ""
INPUT_FILE = "text.txt"


content = load_content()
content_tokens = llm.get_num_tokens(content)
print(f"Content length: {len(content)} chars, {content_tokens} tokens.")
print("Content sample:\n" + content[:200] + "\n\n")

# Keep part of context window for models output.
base_threshold = 0.75*MODEL_CONTEXT_WINDOW

if (content_tokens < base_threshold):
    print("Using summarizer: base")
    summary = summarize_base(llm, content)
else:
    print("Using summarizer: map-reduce")
    summary = summarize_map_reduce(llm, content)

print(f"Content length: {len(summary)} chars, {llm.get_num_tokens(summary)} tokens.")
print("Summary:\n" + summary + "\n\n")


Content length: 7406 chars, 2042 tokens.
Content sample:
Little Red-Cap

Once upon a time there was a dear little girl who was loved
by every one who looked at her, but most of all by her
grandmother, and there was nothing that she would not have
given to t


Using summarizer: base


> Entering new LLMChain chain...
Prompt after formatting:

Write a summary of the following text delimited by tripple backquotes.
Return your response as numbered list which covers the main points of the text.

```Little Red-Cap

Once upon a time there was a dear little girl who was loved
by every one who looked at her, but most of all by her
grandmother, and there was nothing that she would not have
given to the child.  Once she gave her a little cap of red
velvet, which suited her so well that she would never wear
anything else.  So she was always called little red-cap.

One day her mother said to her, come, little red-cap, here
is a piece of cake and a bottle of wine.  Take them to your
grandmother, she

Llama.generate: prefix-match hit



> Finished chain.
Content length: 553 chars, 141 tokens.
Summary:
1. Little Red-Cap is a dear little girl who loves by everyone, especially her grandmother.
2. Her mother sends her with cake and wine to visit her sick grandmother.
3. On the way, she meets a wolf who tries to trick her into leaving the path.
4. The wolf kills her grandmother and takes her form, eating Little Red-Cap when she arrives.
5. A huntsman finds them inside the wolf's stomach and saves them.
6. They fill the wolf's stomach with stones, causing it to collapse and die.
7. Little Red-Cap learns to be cautious and never leaves the path again.


CPU times: total: 30min 29s
Wall time: 7min 40s
